### Jarvis Chatbot with python
- This is my project Jarvis Chatbot with Python which can perform multiple tasks in your `single command`.
- This can perform multiple tasks such as open and close your apps present in your system.

### There are some features my jarvis can perform:
1. Providing time,date and greetings 
2. Controlling in your system volume
3. Opening and closing your application present in your system.
4. You can Search anything from google.
5. Playing songs
6. Searching wikipedia and google for quick information
7. Manage your daily tasks and schedule.
8. Read and Analyze your csv file .
9. Oprn Social Media like Linkedin,Youtube,Instagram 

### Libraries used for
1. ctypes --> system volume control.
2. datetime --> getting current time and day.
3. time --> time related operations.
4. webbrowser --> opening websites in the browser.
5. os --> handling operating system.
6. pyttsx3 --> for text to speech conversion.
7. random --> selecting a random response.
8. wikipedia --> searching wikipedia.
9. pyautogui --> automating mouse and keyword actions.
10. import psutil --> checking system battery percentage.
11. import pandas as pd --> read csv file

In [5]:
import ctypes
import datetime
import time
import webbrowser
import os
import pyttsx3
import random
import wikipedia
import pyautogui
import psutil
import pandas as pd  

# initialize text to speech engine
def initialize_engine():
    engine = pyttsx3.init("sapi5")
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    engine.setProperty('rate', engine.getProperty('rate') - 50)
    engine.setProperty('volume', min(engine.getProperty('volume') + 0.25, 1.0))
    return engine

def speak(text):
    engine = initialize_engine()
    engine.say(text)
    engine.runAndWait()
    
#volume function
def set_volume(level):
    volume = int(level * 0xFFFF)
    ctypes.windll.winmm.waveOutSetVolume(0, volume | (volume << 16))

# increase volume function
def increase_volume():
    set_volume(0.8)
    speak("Volume increased")
    
# decrease volume function
def decrease_volume():
    set_volume(0.3)
    speak("Volume decreased")

# mute volume function
def mute_volume():
    set_volume(0)
    speak("Volume muted")
    
# cal_day
def cal_day():
    day = datetime.datetime.today().weekday() + 1
    return {
        1: "Monday", 2: "Tuesday", 3: "Wednesday",
        4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"
    }.get(day, "Unknown")

# greeting  function
def wish_me():
    hour = datetime.datetime.now().hour
    t = time.strftime("%I:%M %p")
    day = cal_day()
    greeting = "Good morning" if hour < 12 else "Good afternoon" if hour < 18 else "Good evening"
    speak(f"{greeting}, it's {day} and the time is {t}")

# social media function
def social_media(command):
    sites = {
        "linkedin": "https://www.linkedin.com",
        "youtube": "https://www.youtube.com",
        "discord": "https://discord.com/",
        "dei website": "https://www.dei.ac.in/dei/",
        "google classroom": "https://classroom.google.com/u/0/h?hl=en"
    }
    for key in sites:
        if key in command:
            speak(f"Opening {key}")
            webbrowser.open(sites[key])
            return
    speak("No result found")

# schedule function
def schedule():
    week = {
        "monday": "You have a presentation today.",
        "tuesday": "You have a maths exam today.",
        "wednesday": "You have a meeting at 2 PM.",
        "thursday": "You have to practice python in the morning.",
        "friday": "You have a group study at 3 PM.",
        "saturday": "It's a chill!",
        "sunday": "It's a trip day!"
    }
    speak(f"Today's schedule is: {week.get(cal_day().lower(), 'No schedule found.')}")

# display tasks function
def display_tasks():
    if os.path.exists("todo.txt"):
        with open("todo.txt", "r") as file:
            tasks = file.readlines()
        print("Current tasks:", *tasks, sep="\n")
        return tasks
    print("No tasks found.")
    return []

# analyze csv file
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        speak("CSV file loaded successfully!")
        
        print("\n Basic Information:")
        print(df.info())
        speak("Basic information displayed.")

        print("\n summary Statistics:")
        print(df.describe())
        speak("Summary statistics displayed.")

        print("\n **First 5 Rows:**")
        print(df.head())
        speak("Here are the first five rows.")

        print("\n **Missing Values Count:**")
        print(df.isnull().sum())

        print("\n🔹 **Unique Values Count:**")
        print(df.nunique())

        if len(df.select_dtypes(include=['number']).columns) > 1:
            print("\n🔹 **Correlation Matrix:**")
            print(df.corr())

        speak("CSV analysis completed!")
    except Exception as e:
        speak("Error analyzing CSV file.")
        print("Error:", e)

# search wikipedia function
def search_wikipedia(query):
    search_term = query.replace("search wikipedia", "").strip()
    try:
        result = wikipedia.summary(search_term, sentences=2)
        print("Wikipedia Summary:", result)
        speak(result)
    except wikipedia.exceptions.DisambiguationError as e:
        speak("be more specific.")
        print(" error:", e.options)
    except wikipedia.exceptions.PageError:
        speak("No output.")
    except Exception as e:
        speak("error.")
        print("Error:", e)

# close your application
def close_application(app_name):
    for proc in psutil.process_iter():
        if app_name.lower() in proc.name().lower():
            proc.kill()
            speak(f"{app_name} closed successfully.")
            return
    speak(f"{app_name} is not working.")

if __name__ == "__main__":
    speak("Hello, I'm JARVIS")
    wish_me()

    while True:
        try:
            query = input("Enter your command: ").lower()
            speak(f"You said: {query}")

            if "hello" in query:
                speak("Welcome, how may I help you?")
            elif "new task" in query:
                task = query.replace("new task", "").strip()
                if task:
                    with open("todo.txt", "a") as file:
                        file.write(task + "\n")
                    speak(f"Task added: {task}")
            elif "show task" in query:
                tasks = display_tasks()
                if tasks:
                    for task in tasks:
                        speak(task.strip())
                else:
                    speak("You have no task added.")
            elif "remove task" in query:
                tasks = display_tasks()
                if tasks:
                    try:
                        task_number = int(input("Enter your task number to remove: "))
                        if 1 <= task_number <= len(tasks):
                            task_to_remove = tasks.pop(task_number - 1).strip()
                            with open("todo.txt", "w") as file:
                                file.writelines(tasks)
                            speak(f"Task '{task_to_remove}' removed.")
                        else:
                            speak("Invalid task number.")
                    except ValueError:
                        speak("Invalid.")
            elif "search wikipedia" in query:
                search_wikipedia(query)
            elif "search google" in query:
                search_term = query.replace("search google", "").strip()
                webbrowser.open(f"https://www.google.com/search?q={search_term}")
                speak(f"Searching Google for {search_term}")
            elif "volume up" in query:
                increase_volume()
            elif "volume down" in query:
                decrease_volume()
            elif "mute volume" in query:
                mute_volume()
            elif "open" in query:
                app_name = query.replace("open", "").strip()
                pyautogui.press("win")
                pyautogui.typewrite(app_name)
                pyautogui.sleep(2)
                pyautogui.press("enter")
            elif "close" in query:
                close_application(query.replace("close", "").strip())
            elif "play music" in query:
                songs = [
                    "https://www.youtube.com/watch?v=ol4Q1tK0z4g",
                    "https://www.youtube.com/watch?v=98IJZ__ws_g"
                ]
                webbrowser.open(random.choice(songs))
                speak("Playing music")
            elif "schedule" in query:
                schedule()
            elif "analyze csv" in query:
                analyze_csv(input("Enter CSV file path: "))
            elif "exit" in query:
                speak("Exiting JARVIS. Goodbye!")
                break
        except KeyboardInterrupt:
            speak("Exiting JARVIS. Goodbye!")
            break


Enter your command:  play music
Enter your command:  open excel
Enter your command:  close excel
Enter your command:  analyze csv
Enter CSV file path:  IPL_Matches_2008_2022.csv



 Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               950 non-null    int64  
 1   City             899 non-null    object 
 2   Date             950 non-null    object 
 3   Season           950 non-null    object 
 4   MatchNumber      950 non-null    object 
 5   Team1            950 non-null    object 
 6   Team2            950 non-null    object 
 7   Venue            950 non-null    object 
 8   TossWinner       950 non-null    object 
 9   TossDecision     950 non-null    object 
 10  SuperOver        946 non-null    object 
 11  WinningTeam      946 non-null    object 
 12  WonBy            950 non-null    object 
 13  Margin           932 non-null    float64
 14  method           19 non-null     object 
 15  Player_of_Match  946 non-null    object 
 16  Team1Players     950 non-null    object 


Enter your command:  exit
